In [ ]:
# default_exp generate_report

# generate_report

> This takes two branches and compares them for differences. Any differences are made into a summay html page, and details pages for each table/file that has difference in it. It also saves a copy of the files that are different if required in the report directory


In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *
from nbdev import *

In [ ]:
#export

#C--------------------------------------------------------------------------
#C  Program : initalise_repository.py
#C 
#C  Generates a difference report between two branches (servers) in the repository
#C 
#C  Copyright: 3ideas.co.uk  
#C
#C  See https://github.com/3ideas/config_tracker
#C--------------------------------------------------------------------------

import argparse
from os import listdir
from os.path import isfile, join,exists
import os
import sys
import datetime
import shutil

from subprocess import call, check_output


In [ ]:
#export
verbose = False
show_progress = True

def get_output_from_cmd(cmd):
    """
    Get the output from a command
    """
    try:
        if verbose:
            print('RUN: %s' %cmd)
        output = check_output(cmd, shell=True)
        return output.decode("utf-8")
    except Exception as e:
        print(e)
        return None


def run_subprocess(command):
    """
    Run the subprocess
    """
    try:

        retcode = call(command, shell=True)
        if verbose:
            print("Command: %s\nReturned: %s" %(command,retcode))
        if retcode < 0:
            sys.stderr.write("Command: %s, was terminated by signal: %s\n" % (command, retcode))
        elif retcode > 0:
            sys.stderr.write("Command: %s, returned: %s \n"% (command, retcode))
    except OSError as e:
        sys.stderr.write("Execution failed: %s \n"% e)
    return retcode


In [ ]:
#export
############################################################################################
#
# Git functions
#
############################################################################################

class GitRepositoryReader:
    
    def __init__(self, repository, output_dir, branch1, branch2=None):
        self.repository = repository
        self.branch1 = branch1
        self.branch2 = branch2
        self.output_dir = output_dir
        
    def run_git_command(self, command,splitlines = True):
        """
        Run a git command
        """
        result = get_output_from_cmd('cd %s && git %s' % (self.repository, command))
        if result is not None and splitlines:
            return result.splitlines()
        return result
        
    def set_branch1(self, branch1):
        self.branch1 = branch1

    def get_list_of_files_differences_between_branches(self):
        """ Get the list of file difference between 2 git branches
        """
        file_list = self.run_git_command("diff --name-only %s %s" %(self.branch1,self.branch2))
        return file_list

    def get_list_of_file_changes(self,filename):
        """ get the list of changes to `filename` in a repository,
        returns a list of tubles with (git commit hash, commit details(date), datetime)"""

        change_list = []
    
        format_str='%h %s'
        lines = self.run_git_command('log %s --pretty=format:"%s" -- %s' % (self.branch1,format_str,filename))
        for line in lines:
            line = line.strip()
            fields = line.split()
            date = parse_commitstr_to_datetime(fields[1])
            change_list.append([fields[0],fields[1],date])
    
        return change_list # TODO make this a structure!

    def get_filename_ref_for(self, git_reference, filename):
        """ Given a repository reference (branch/commit), return the fullpath to the file in the output_dir."""
        part1        = os.path.join(self.output_dir, 'original_files')
        part2         = os.path.join(part1, git_reference)
        filename_output_path = os.path.join(part2, filename)
        return filename_output_path

    def get_all_filenames_in_branch(self,branch=None):
        """ Returns a list of all filenames in the database and files directories in a branch/commit"""
        
        if branch is None:
            branch = self.branch1
    
        lines = self.run_git_command('ls-tree -r --name-only %s' %(branch))
        filenames = []
        for line in lines:
            if line.startswith('database') or line.startswith('files') or line == 'last_updated.md':
                filenames.append(line)
        return filenames

    def copy_file_to_output(self,filename, git_reference=None): #     git_get_file:
        """ retreive filename from repository in the output_dir 
        the ref can be a branchname or a commit hash (long or short)
        this enable us to retrieve files from commits without having to checkout
        the file
        
        returns the full pathname of the file copied, returns None if unable to get the file"""
        
        # if no reference is given default to branch1
        if git_reference is None: 
            git_reference = branch1 
    
        filename_output_location = self.get_filename_ref_for(git_reference,filename)
        #print('output filename: %s' % filename_output_location)
        # if the file already exists in the output dir do not copy it again
        # this saves us getting the same file over and over.
        if os.path.isfile(filename_output_location):
            return filename_output_location
    
        if not mkdir_for_fullpath(filename_output_location):
            return None 
    
        #  cd ../config_repository && git show d08b328:database/ALARM_DEFINITION.csv    
        file_contents = self.run_git_command('show %s:%s' %(git_reference,filename),splitlines = False)
        if file_contents is None:
            return None
        hs = open(filename_output_location, 'w')
        if len(file_contents) !=0: # can't write 0 contents to file ...
            hs.write(file_contents)
        hs.close()
        
        return filename_output_location
    


In [ ]:
#export

def parse_commitstr_to_datetime(string):
    """
    Parse the string  to a timestamp, tries to interperate the comment as a date.
    TODO: use the commit date itself !
    """
    if len(string) < 8:
        return None

    try:
        year = int(string[0:4])
        month = int(string[4:6])
        date = int(string[6:8])
        dt = datetime.datetime(year=year, month=month, day=date)
    except:
        return None
    return dt

############################################################################################
#
# file functions
#
############################################################################################   

def copyfile_fullpath(file_name,dest_dir):
    """ Copy the file to the destination dir in the same directory structure as its original path"""

    
    # TODO : use     if not mkdir_for_fullpath(filename_output_location):
    directory_name = os.path.dirname(file_name)
    base_name = os.path.basename(file_name)

    if len(directory_name) >0 and directory_name[0] == '/':
        directory_name = directory_name[1:]

    dir_to_create = os.path.join(dest_dir,directory_name)

    copy_name = file_name
    if copy_name[0] == '/':
        copy_name = copy_name[1:]

    dest_full_name = os.path.join(dest_dir,copy_name)

    if not os.path.isdir(dir_to_create):
        try:
            os.makedirs(dir_to_create)
        except OSError as error:
            sys.stderr.write("Directory '%s' cannot be created, skipping file: %s\n" % (error,file_name))
            return
    try:
        shutil.copyfile(file_name,dest_full_name)
    except Exception as e:
        sys.stderr.write("File cannot be copied to dest : %s, skipping file: %s\n" % (e,file_name))

        

def mkdir_for_fullpath(filename_output_location):
    
    path, file = os.path.split(filename_output_location)
    
    if not os.path.isdir(path):
        try:
            os.makedirs(path)
        except OSError as error:
            sys.stderr.write("Directory '%s' cannot be created, skipping file: %s\n" % (error,path))
            return False
    return True

#def copy_branch_to_output(repository, output_dir, branch, file_list):
#    """
#    Copy the files to the output directory
#    """
#    cwd = os.getcwd()
#    os.chdir(repository)
#    # checkout the branch
#    switch_to_branch(repository, branch)
#    
#    dest_dir =  os.path.join(output_dir,branch)
#    if not os.path.isdir(dest_dir):
#        os.mkdir(dest_dir)
#    # copy the files
#    for file_name in file_list:
#        copyfile_fullpath(file_name,dest_dir)
#    os.chdir(cwd)
    
def remove_files_from_output_branch( output_dir, branch, files):
    """
    remove  files from the output directory
    """
    dest_dir =  os.path.join(output_dir,branch)
    print('Would delete: %s'% dest_dir)
    #shutil.rmtree(dest_dir) 
    
    
def remove_files_from_output(output_dir, branch1,branch2, file_list):
    remove_files_from_output_branch(output_dir,branch1,file_list)
    remove_files_from_output_branch(output_dir,branch2,file_list)

#def copy_files_to_output(repository,output_dir, branch1,branch2, files_with_differences):
#    """
#    Copy the files to the output directory
#    """
#    copy_branch_to_output(repository,output_dir,branch1,files_with_differences)
#    copy_branch_to_output(repository,output_dir,branch2,files_with_differences)

#def copy_all_file_versions_to_output(repository,output_dir,commit_log,filename):
#    """ copy all versions of a file to the output"""
#    
#    for commit_entry in commit_log:
#        commit_hash=commit_entry[0]
#        git_get_file(repository, commit_hash,filename,output_dir )
   

    
def get_report_name_refs(repository_filename, 
                         change_ref=None, 
                         report_base='',
                         output_dir=None,
                         add_report_base_to_link=False,
                         link_ref_strip_dir=False):
    """ given a repository filename, returns the name stripped in 3 forms
    the visible_name, the link_ref and the filename it should be stored under
    also works for non-repository names ...
    """
    visible_name = repository_filename
    link_ref = repository_filename
    filename = repository_filename
    base = repository_filename

    if repository_filename.startswith("database/"):
        
        base = repository_filename.replace("database/", "")
        base = base.replace(".csv", "")
        visible_name = base
        filename = repository_filename.replace(".csv", "")
        
            
    if repository_filename.startswith("files/"):
        base = repository_filename.replace("files/", "")   
        visible_name = '/'+base

    if link_ref_strip_dir:
        _, link_ref = os.path.split(filename)
        #link_ref = base
    else:
        link_ref = filename
    
    if change_ref is not None:
        filename = filename + '_' + change_ref
        link_ref = link_ref + '_' + change_ref
    
       
    link_ref = link_ref + '.html'
    filename = filename + '.html'
    
    if add_report_base_to_link:
        link_ref = os.path.join(report_base,link_ref)
    filename = os.path.join(report_base,filename)
    if output_dir is not None:
        filename = os.path.join(output_dir,filename)
    return (visible_name,link_ref, filename)
 
def make_output_directories(output_dir,base):
    """make the directories required for the reports"""
    database_dir = os.path.join(output_dir,'database')
    files_dir = os.path.join(output_dir,'files')
    orig_dir = os.path.join(output_dir,'original_files')
    
    print('database output dir: %s'%database_dir)
    
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    if not os.path.isdir(database_dir):
        os.mkdir(database_dir)
    if not os.path.isdir(files_dir):
        os.mkdir(files_dir)  
    if not os.path.isdir(orig_dir):
        os.mkdir(orig_dir)  
        
    

        

In [ ]:
#hide
visible_name,link_ref,filename = get_report_name_refs('database/table_name.csv','change_ref','report_type')
test_eq(visible_name,'table_name')
test_eq(link_ref,'database/table_name_change_ref.html')
test_eq(filename,'report_type/database/table_name_change_ref.html')


In [ ]:
#hide
visible_name,link_ref,filename = get_report_name_refs('files/file_name.csv','change_ref','report_type')
test_eq(visible_name,'/file_name.csv')
test_eq(link_ref,'files/file_name.csv_change_ref.html')
test_eq(filename,'report_type/files/file_name.csv_change_ref.html')


In [ ]:
#hide
visible_name,link_ref,filename = get_report_name_refs('database/table_name.csv')
test_eq(visible_name,'table_name')
test_eq(link_ref,'database/table_name.html')
test_eq(filename,'database/table_name.html')

In [ ]:
#hide
visible_name,link_ref,filename = get_report_name_refs('branch_name')
test_eq(visible_name,'branch_name')
test_eq(link_ref,'branch_name.html')
test_eq(filename,'branch_name.html')

In [ ]:
#hide
visible_name,link_ref,filename = get_report_name_refs('branch_name',
                                                      change_ref='hello',
                                                      output_dir='temp',
                                                      report_base='server1')
test_eq(visible_name,'branch_name')
test_eq(link_ref,'branch_name_hello.html')
test_eq(filename,'temp/server1/branch_name_hello.html')

In [ ]:
#hide

visible_name,link_ref,filename = get_report_name_refs('database/branch_name.csv',
                                                      change_ref='hello',
                                                      output_dir='temp',
                                                      report_base='server1',
                                                     add_report_base_to_link=True)
test_eq(visible_name,'branch_name')
test_eq(link_ref,'server1/database/branch_name_hello.html')
test_eq(filename,'temp/server1/database/branch_name_hello.html')


In [ ]:
#hide
visible_name,link_ref,filename = get_report_name_refs('database/branch_name.csv',
                                                      change_ref='hello',
                                                      output_dir='temp',
                                                      report_base='server1',
                                                     link_ref_strip_dir=True)
test_eq(visible_name,'branch_name')
test_eq(link_ref,'branch_name_hello.html')
test_eq(filename,'temp/server1/database/branch_name_hello.html')

In [ ]:
#hide
visible_name,link_ref,filename = get_report_name_refs('files/users/bt/change_name/dummy.txt',
                                                      change_ref='hello',
                                                      output_dir='temp',
                                                      report_base='server1',
                                                     link_ref_strip_dir=True)
test_eq(visible_name,'/users/bt/change_name/dummy.txt')
test_eq(link_ref,'dummy.txt_hello.html')
test_eq(filename,'temp/server1/files/users/bt/change_name/dummy.txt_hello.html')

In [ ]:
#export

def get_relative_path(repository_filename):
    """ returns the path back to the base dir, from the given filename"""
    
    path = repository_filename
    c = 0
    while path != '':
        c += 1
        path,_ = os.path.split(path)
    
    rel_path = ''
    while c > 0:
        rel_path += '../'
        c -= 1
    return rel_path



def copy_style_and_bootstrap(src,dst):
    """ copy the required styles to dst. 
    copies all the files from src to dst, if dst already exists it does nothing
    """
    dst = os.path.join(dst,'styles')
    if not os.path.isdir(dst):    
        shutil.copytree(src, dst)    

        
def bootstrap_navbar(relative_path='',title='',description=''):
    return """<!-- Image and text -->
<nav class="navbar navbar-light bg-light">
  <a class="navbar-brand" href="#">
    <img src="%sstyles/logo.svg"  height="30" class="d-inline-block align-top" alt="">
    &nbsp;&nbsp;%s : &nbsp;&nbsp;%s
  </a>
</nav>""" % (relative_path,title,description)


def header(relative_path='',description=''):
    h = '<!doctype html>\n'
    h += '<html lang="en">\n'
    h += '<head>\n'
    h += '<title>%s</title>\n' % description
    h += '<link rel="stylesheet" href="%sstyles/bootstrap-4.0.0-dist/css/bootstrap.css">\n' %relative_path
    h += '<link rel="stylesheet" href="%sstyles/style.css">\n' % relative_path
    h += '</head>\n' 
    
    return h

default_footer='<a href="https://github.com/3ideas/config_tracker">Config Tracker</a> written by <a href="https://3ideas.co.uk/">3ideas.co.uk</a>'

def footer(footer_text=default_footer):
    return """<footer class="footer">
            <div class="container">
            <span class="text-muted">%s</span>
            </div>
            </footer>""" % footer_text

def page(relative_path='',description='',content='',title='',footer_text=default_footer):
    p = header(relative_path=relative_path,description=description)
    p += '<body>\n'
    p += bootstrap_navbar(relative_path=relative_path,title=title,description=description)
    p += content
    p += footer(footer_text)
    p += '</body>\n'
    p += '</html>\n'
    return p
  
    

def diff2html_template(relative_path='',description='',title='',footer_text=default_footer):
    p = header(relative_path=relative_path,description=description)
    # TODO add relative local stylesheet !!! use relative path!
    p += """<link rel="stylesheet" href="%sstyles/diff2html.css" />
<!--diff2html-css-->
<!--diff2html-js-ui-->
<script>
  document.addEventListener('DOMContentLoaded', () => {
    const targetElement = document.getElementById('diff');
    const diff2htmlUi = new Diff2HtmlUI(targetElement);
    //diff2html-fileListToggle
    //diff2html-synchronisedScroll
    //diff2html-highlightCode
  });
    
</script>""" % relative_path
    p += '<body>\n'
    p += bootstrap_navbar(relative_path=relative_path,title=title,description=description)
    p += """<div id="diff">
  <!--diff2html-diff-->
</div>"""
    p += footer(footer_text)
    p += '</body>\n'
    p += '</html>\n'
    return p 
    
    
    

In [ ]:
#export
############################################################################################
#
# report functions
#
############################################################################################

#
# General difference function ... 
# used for all the report differences
#
def generate_difference_report_page(r,repository_filename,branch1, branch2,output_file_name , description,title):
    """ generate a difference report for `file`. 
    This function assumes the files have already been copied to each of the branch sub directories
    This will work for comparing between systems or comparing history of the file"""
    
    visible_name,_,_ = get_report_name_refs(repository_filename)
    full_description = ' %s : %s' % (visible_name,description)  # TODO escape any special chars
    
    relative_path = get_relative_path(repository_filename)
    
    mkdir_for_fullpath(output_file_name) # TODO check for errors
    # 
    # get the files to compare from the repository
    #
    
    file1 = r.copy_file_to_output(repository_filename,git_reference=branch1)  
    file2 = r.copy_file_to_output(repository_filename,git_reference=branch2)
    # TODO check for file1 or file2 being None !
    if file1 is None or file2 is None:
        
        content = '<p>Unable to generate report for: %s to %s </p>' % (branch1,branch2)
        if file1 is None:
            content += "<p>File: %s does not exist for: %s </p>" %(repository_filename,branch1)
        if file2 is None:
            content += "<p>File: %s does not exist for: %s </p>" %(repository_filename,branch2)
        full_page = page(relative_path=relative_path,
                         description=description,
                         content=content, 
                         title=title
                        )
        hs = open(output_file_name, 'w')
        hs.write(full_page)
        hs.close()
        return
    
    
    # if its a database file then its a csv diff
    if repository_filename.endswith('.csv'):
        if show_progress:
            print("Generating csv comparison for : %s  between: %s, %s" %(visible_name,branch1,branch2))
        
        tmp_diff_file = os.path.join(r.output_dir,'diff_DELETEME.csv' )
        run_subprocess('daff diff %s %s  >%s'%(file1,file2,tmp_diff_file))
        diff_table_html = get_output_from_cmd('daff.py render --fragment %s'% tmp_diff_file)
        run_subprocess('rm %s'% tmp_diff_file)
        
        diff_table_html = diff_table_html.replace('<table>','<table class="table table-sm">',1)

        content =  "<div class='highlighter'>\n"
        content += '<div class="table-responsive">\n'
        content += '<div class="tableFixHead">\n'
        content += '<div class="h-100 d-inline-block">\n'
        content += diff_table_html
        content += "</div>\n"
        content += "</div>\n"
        content += "</div>\n"
        content += "</div>\n"
        full_page = page(relative_path=relative_path,
                         description=description,
                         content=content, 
                         title=title
                        )
        
        hs = open(output_file_name, 'w')
        hs.write(full_page)
        hs.close()
    else:
        if show_progress:
            print("Generating    comparison for : %s  between: %s, %s" %(visible_name,branch1,branch2))
        # generate side by side diff
        if os.path.exists(output_file_name):
            os.remove(output_file_name)
        template = diff2html_template(relative_path=relative_path,
                         description=description, 
                         title=title
                        )
        hs = open('template.html', 'w')
        hs.write(template)
        hs.close()
        run_subprocess('diff2html -s side --hwt template.html -F %s %s %s '%(output_file_name,file1,file2))
        run_subprocess('rm template.html')
        
    

In [ ]:
#export
###################
#
# Branch comparison report 
#
####################

class BranchComparisonReport:
    # All files are stored under the branch_name 
    
    def __init__(self, repository, branch1,branch2, output_dir,title='Config Tracker',styles_src=None):
        self.repository = repository
        self.branch1 = branch1
        self.branch2 = branch2
        self.output_dir = output_dir
        self.title = title
        self.r = GitRepositoryReader(repository,output_dir,branch1,branch2)
        self.file_list = self.r.get_list_of_files_differences_between_branches()
        if styles_src is not None:
            copy_style_and_bootstrap(styles_src,output_dir)

            
    def generate_report(self):
        """ this generates the index page and all the comparisons for files accross 2 branches."""
        
        self.index_page()
        self.difference_reports()        
               
    def get_change_ref(self):
        return self.branch1+'_'+self.branch2

    def index_page(self):
        """ Generate the index page for all changes between 2 branches/systems"""
        description = 'Differences between %s and %s' %(self.branch1,self.branch2)
        content = "<table><tr><th>Table Name</th></tr>\n"
        for repository_filename in self.file_list:
            change_ref = self.get_change_ref()
            visible_name,link_name,_ = get_report_name_refs(repository_filename,
                                                        change_ref=change_ref,
                                                        report_base=change_ref,
                                                       output_dir=self.output_dir,
                                                       add_report_base_to_link=True)
        
            content += '<tr><td><a href="%s">%s</a></td></tr>\n' %(link_name,visible_name)
        content  += "</table>\n"

        full_page = page(relative_path='',
                         description=description,
                         content=content, 
                         title=self.title
                        )        

        index_file = os.path.join(self.output_dir,'index_%s.html' % self.get_change_ref())
        hs = open(index_file, 'w')
        hs.write(full_page)
        hs.close()
    
    #
    # branch comparison report, for 2 branches generate report of the changes between them
    #
    def difference_reports(self): 
        """ generate a difference report for each file in the repository in the `file_list` """
    
        for repository_filename in self.file_list:
            change_ref = self.get_change_ref()
            visible_name,_,output_filename = get_report_name_refs(repository_filename,
                                                   change_ref=change_ref,
                                                   report_base=change_ref,
                                                  output_dir=self.output_dir)
            description = 'Differences between %s and %s :   %s' %(self.branch1,self.branch2,visible_name)
            generate_difference_report_page(self.r,
                                            repository_filename,
                                            self.branch1,
                                            self.branch2,
                                            output_filename,
                                            title = self.title,
                                            description = description)
        




In [ ]:
#export

###################
#
# File Change History on a branch
#
####################    

class FileChangeHistoryReportForBranch:
    # All files are stored under the branch_name 
    
    def __init__(self, repository, branch, output_dir,title='Config Tracker',styles_src=None):
        self.repository = repository
        self.branch = branch
        self.output_dir = output_dir
        self.title = title
        self.r = GitRepositoryReader(repository,output_dir,branch)
        self.filename_list = self.r.get_all_filenames_in_branch()
        self.report_desc_name='change_history'
        
        if styles_src is not None:
            copy_style_and_bootstrap(styles_src,output_dir)
        
        #make_output_directories(output_dir,branch) 
            
    def generate_report(self):
        self.index_page()
        for repository_filename in self.filename_list:
            self.reports_for_file(repository_filename)  
            
    def format_date(self,date):
        """ Formats the date for display next to change history"""
        return date.strftime("%A %d. %B %Y")
        
    def index_page(self):
        """ File history index for the given server"""
        description = 'Change history for: %s' % self.branch 
        
        content = "<table><tr><th>File/tablename</th></tr>\n"
        for repository_filename in self.filename_list:
            visible_name,report_link,_ = get_report_name_refs(repository_filename,
                                                              change_ref=self.report_desc_name,
                                                              report_base=self.branch,
                                                              add_report_base_to_link = True)
            content += '<tr><td><a href="%s">%s</a></td></tr>\n' %(report_link,visible_name)
        content += "</table>\n"
        
        _,_,index_filename = get_report_name_refs(self.branch,
                                                  change_ref=self.report_desc_name,
                                                  output_dir=self.output_dir)
        
        full_page = page(relative_path='',
                         description=description,
                         content=content, 
                         title=self.title
                         )       
        
        mkdir_for_fullpath(index_filename)  
        hs = open(index_filename, 'w')
        hs.write(full_page)
        hs.close()    
        
    #
    # file history report, generate report for all changes made to a file
    #        
    def reports_for_file(self, repository_filename):
        
        #
        # Get the change history for the file (in this branch)
        commit_log = self.r.get_list_of_file_changes(repository_filename)
    
        self.file_index_page(repository_filename, commit_log)
   
        if len(commit_log) <= 1:
            return # No history available
        else:
            first = True
            for change_entry in commit_log:
                # a change_entry is a tuple with the following format:
                # (git commit hash, commit details(date), datetime)
                commit_hash = change_entry[0]
                commit_ref = change_entry[1]
                datetime = change_entry[2]

                if not first: 
                    visible_name,_,output_filename = get_report_name_refs(repository_filename,
                                                      #change_ref=commit_ref,
                                                      change_ref=commit_hash,
                                                      output_dir=self.output_dir,
                                                      report_base=self.branch)
                    description = 'Changes made on: %s  to: %s' % (self.format_date(datetime),visible_name)
                    generate_difference_report_page(self.r,repository_filename,
                                                commit_hash,prev_commit_hash,
                                                output_filename,
                                                description = description,
                                                title=self.title)
                else:
                    first = False
                
                prev_commit_hash = change_entry[0]
                prev_commit_ref = change_entry[1]
                prev_datatime = change_entry[2]
    
    def file_index_page(self,repository_filename, commit_log):
        """ generates a page that shows all the changes to a file and the dates they occurred on. 
        If there is only one commit in the files history then no changes have occurred since it was
        first commited.
        """
        
        visible_name,_,output_filename = get_report_name_refs(repository_filename,
                                                      change_ref=self.report_desc_name, 
                                                      output_dir=self.output_dir,
                                                      report_base=self.branch)        
        
        description = 'Change history for %s' % visible_name
 
        content = ''
    
        if len(commit_log) <= 1:
            content += '<p> No changes recorded </p>' 
        else:
            content += "<table><tr><th>Date of change</th></tr>\n"
            first = True
            for change_entry in commit_log:
                if first: # skip the first commit 
                    first = False
                    continue 
                _,diff_file_link,_ = get_report_name_refs(repository_filename,
                                                      #change_ref=change_entry[1],
                                                          change_ref=change_entry[0],
                                                         link_ref_strip_dir=True)
                date_str = self.format_date(change_entry[2])
                content += '<tr><td><a href="%s">%s</a></td></tr>\n' %(diff_file_link,date_str)

            content += "</table>\n"
            
            
        full_page = page(relative_path=get_relative_path(repository_filename),
                         description=description,
                         content=content, 
                         title=self.title
                         )   

        mkdir_for_fullpath(output_filename)    
        hs = open(output_filename, 'w')
        hs.write(full_page)
        hs.close()
                

In [ ]:
#hide
repository = '../config_repository'
b1 = 'server1'
b2 = 'server2'
output_dir = '../config_reports'
styles_src = './scripts/styles'

In [ ]:
#hide
report = BranchComparisonReport(repository, b1,b2, output_dir,styles_src=styles_src)
report.generate_report()

Generating csv comparison for : ACTION_APPEARANCES  between: server1, server2
Generating csv comparison for : ACTION_DEFINITIONS  between: server1, server2
Generating csv comparison for : ALARM_DEFINITION  between: server1, server2


fatal: path 'database/ALARM_PRIORITIES.csv' does not exist in 'server1'
fatal: path 'database/ALARM_TYPE.csv' does not exist in 'server1'


Command 'cd ../config_repository && git show server1:database/ALARM_PRIORITIES.csv' returned non-zero exit status 128.
Command 'cd ../config_repository && git show server1:database/ALARM_TYPE.csv' returned non-zero exit status 128.
Generating csv comparison for : COMPONENT_CLASS_DEFN  between: server1, server2
Generating csv comparison for : COMPONENT_FUNCTION  between: server1, server2
Generating csv comparison for : COMPONENT_NAME_RULE  between: server1, server2
Generating csv comparison for : COMPUTER_EQPT  between: server1, server2
Generating csv comparison for : CONTROL_ZONE  between: server1, server2
Generating csv comparison for : COUNTER  between: server1, server2
Generating csv comparison for : CR_CALL_CATEGORY  between: server1, server2
Generating csv comparison for : CUSTOMER_LOG_CAT  between: server1, server2
Generating csv comparison for : DOCUMENT_CLASS_DEFN  between: server1, server2
Generating csv comparison for : ENVIRONMENT_LIST  between: server1, server2
Generating c

fatal: path 'database/OUTPUT.html' exists on disk, but not in 'server2'


Command 'cd ../config_repository && git show server2:database/OUTPUT.html' returned non-zero exit status 128.
Generating csv comparison for : OZ_OPERATING_ZONE  between: server1, server2
Generating csv comparison for : PAGE_MAPPING  between: server1, server2
Generating csv comparison for : PROCESS_TABLE  between: server1, server2
Generating csv comparison for : PROCESS_VIEW  between: server1, server2
Generating csv comparison for : RESOURCE_AUTH_CATEGORY  between: server1, server2
Generating csv comparison for : RESOURCE_CLASSIFICATION  between: server1, server2
'utf-8' codec can't decode byte 0xf1 in position 10703: invalid continuation byte
'utf-8' codec can't decode byte 0xa0 in position 660728: invalid start byte
Generating csv comparison for : RESOURCE_LOOKUP  between: server1, server2
Generating csv comparison for : RESOURCE_NOTIFICATION_METHODS  between: server1, server2
Generating csv comparison for : STORM_PARAMETERS  between: server1, server2
Generating csv comparison for : S

fatal: path 'server2.md' does not exist in 'server1'


In [ ]:
#hide
report = FileChangeHistoryReportForBranch(repository,b1,output_dir,styles_src=styles_src)
report.generate_report()

Generating csv comparison for : ACTION_APPEARANCES  between: facda59, 1785342
Generating csv comparison for : ACTION_APPEARANCES  between: 4c535f0, facda59
Generating csv comparison for : ACTION_APPEARANCES  between: 865f379, 4c535f0
Generating csv comparison for : ACTION_APPEARANCES  between: 9fe248c, 865f379
Generating csv comparison for : ACTION_APPEARANCES  between: 66c69c4, 9fe248c
Generating csv comparison for : ACTION_APPEARANCES  between: d08b328, 66c69c4
Generating csv comparison for : ACTION_APPEARANCES  between: 4186c09, d08b328
Generating csv comparison for : ACTION_DEFINITIONS  between: 575a5df, facda59
Generating csv comparison for : ACTION_DEFINITIONS  between: 3946dc4, 575a5df
Generating csv comparison for : ACTION_DEFINITIONS  between: 3632869, 3946dc4
Generating csv comparison for : ACTION_DEFINITIONS  between: 0e4fa03, 3632869
Generating csv comparison for : ACTION_DEFINITIONS  between: 2bfe901, 0e4fa03
Generating csv comparison for : ACTION_DEFINITIONS  between: 9fe

Generating csv comparison for : COUNTER  between: 70419a9, daacc37
Generating csv comparison for : COUNTER  between: 5e97581, 70419a9
Generating csv comparison for : COUNTER  between: 77ad045, 5e97581
Generating csv comparison for : COUNTER  between: 8165b73, 77ad045
Generating csv comparison for : COUNTER  between: 1e719fd, 8165b73
Generating csv comparison for : COUNTER  between: b1834a9, 1e719fd
Generating csv comparison for : COUNTER  between: 8608335, b1834a9
Generating csv comparison for : COUNTER  between: a29cf59, 8608335
Generating csv comparison for : COUNTER  between: 9d8da35, a29cf59
Generating csv comparison for : COUNTER  between: ef69bc1, 9d8da35
Generating csv comparison for : COUNTER  between: d55cd7f, ef69bc1
Generating csv comparison for : COUNTER  between: 15f6640, d55cd7f
Generating csv comparison for : COUNTER  between: 3a5a7eb, 15f6640
Generating csv comparison for : COUNTER  between: ef53f17, 3a5a7eb
Generating csv comparison for : COUNTER  between: ed43977, ef5

Generating csv comparison for : COUNTER  between: 19305c5, 3b7692c
Generating csv comparison for : COUNTER  between: 6596b03, 19305c5
Generating csv comparison for : COUNTER  between: 9b2e21f, 6596b03
Generating csv comparison for : COUNTER  between: 574cd9b, 9b2e21f
Generating csv comparison for : COUNTER  between: a941e0d, 574cd9b
Generating csv comparison for : COUNTER  between: 8aaaa76, a941e0d
Generating csv comparison for : COUNTER  between: bf9aa90, 8aaaa76
Generating csv comparison for : COUNTER  between: d5f2cf5, bf9aa90
Generating csv comparison for : COUNTER  between: acb59dd, d5f2cf5
Generating csv comparison for : COUNTER  between: d62f888, acb59dd
Generating csv comparison for : COUNTER  between: 0d5cffa, d62f888
Generating csv comparison for : COUNTER  between: 447394d, 0d5cffa
Generating csv comparison for : COUNTER  between: 46a5a51, 447394d
Generating csv comparison for : COUNTER  between: 59245aa, 46a5a51
Generating csv comparison for : COUNTER  between: cf88acd, 592

Generating csv comparison for : INC_GROUPS  between: 9fe248c, a466db6
Generating csv comparison for : INC_GROUPS  between: 4186c09, 9fe248c
Generating csv comparison for : INC_LOG_CATEGORY  between: 1e1c64d, 1785342
Generating csv comparison for : INC_LOG_CATEGORY  between: 2bfe901, 1e1c64d
Generating csv comparison for : INC_LOG_CATEGORY  between: a466db6, 2bfe901
Generating csv comparison for : INC_LOG_CATEGORY  between: 9fe248c, a466db6
Generating csv comparison for : INC_LOG_CATEGORY  between: 1e1d0b3, 9fe248c
Generating csv comparison for : INC_LOG_CATEGORY  between: 6596b03, 1e1d0b3
Generating csv comparison for : INC_LOG_CATEGORY  between: 574cd9b, 6596b03
Generating csv comparison for : INC_LOG_CATEGORY  between: 49aae99, 574cd9b
Generating csv comparison for : INC_LOG_CATEGORY  between: 66c69c4, 49aae99
Generating csv comparison for : INC_LOG_CATEGORY  between: d08b328, 66c69c4
Generating csv comparison for : INC_LOG_CATEGORY  between: 4186c09, d08b328
Generating csv compariso

Generating csv comparison for : PROCESS_TABLE  between: 3946dc4, f322cf1
Generating csv comparison for : PROCESS_TABLE  between: f983297, 3946dc4
Generating csv comparison for : PROCESS_TABLE  between: 9fe248c, f983297
Generating csv comparison for : PROCESS_TABLE  between: 59245aa, 9fe248c
Generating csv comparison for : PROCESS_TABLE  between: 66c69c4, 59245aa
Generating csv comparison for : PROCESS_TABLE  between: 4186c09, 66c69c4
Generating csv comparison for : PROCESS_VIEW  between: e0fcdda, 8de908a
Generating csv comparison for : PROCESS_VIEW  between: 39ae612, e0fcdda
Generating csv comparison for : PROCESS_VIEW  between: facda59, 39ae612
Generating csv comparison for : PROCESS_VIEW  between: 575a5df, facda59
Generating csv comparison for : PROCESS_VIEW  between: f861e43, 575a5df
Generating csv comparison for : PROCESS_VIEW  between: eb749fb, f861e43
Generating csv comparison for : PROCESS_VIEW  between: f322cf1, eb749fb
Generating csv comparison for : PROCESS_VIEW  between: 394

Generating csv comparison for : RESOURCE_DETAILS  between: 22256d7, 228f0f5
Generating csv comparison for : RESOURCE_DETAILS  between: d5d7615, 22256d7
Generating csv comparison for : RESOURCE_DETAILS  between: 566dc5e, d5d7615
Generating csv comparison for : RESOURCE_DETAILS  between: 1dbd4cf, 566dc5e
Generating csv comparison for : RESOURCE_DETAILS  between: 3f2b31b, 1dbd4cf
Generating csv comparison for : RESOURCE_DETAILS  between: c13427f, 3f2b31b
Generating csv comparison for : RESOURCE_DETAILS  between: 661e809, c13427f
Generating csv comparison for : RESOURCE_DETAILS  between: ce8bc2b, 661e809
Generating csv comparison for : RESOURCE_DETAILS  between: 3f14622, ce8bc2b
Generating csv comparison for : RESOURCE_DETAILS  between: 49aae99, 3f14622
Generating csv comparison for : RESOURCE_DETAILS  between: ffb3b29, 49aae99
Generating csv comparison for : RESOURCE_DETAILS  between: d6363b2, ffb3b29
Generating csv comparison for : RESOURCE_DETAILS  between: 66c69c4, d6363b2
Generating c

Generating    comparison for : last_updated.md  between: 4fcc5b4, a7b0258
Generating    comparison for : last_updated.md  between: 7b0e3b7, 4fcc5b4
Generating    comparison for : last_updated.md  between: ba7ac90, 7b0e3b7
Generating    comparison for : last_updated.md  between: 39fb2bd, ba7ac90
Generating    comparison for : last_updated.md  between: 002e9dd, 39fb2bd
Generating    comparison for : last_updated.md  between: 27abd89, 002e9dd
Generating    comparison for : last_updated.md  between: fe438e9, 27abd89
Generating    comparison for : last_updated.md  between: daacc37, fe438e9
Generating    comparison for : last_updated.md  between: 70419a9, daacc37
Generating    comparison for : last_updated.md  between: 5e97581, 70419a9
Generating    comparison for : last_updated.md  between: 77ad045, 5e97581
Generating    comparison for : last_updated.md  between: 8165b73, 77ad045
Generating    comparison for : last_updated.md  between: 1e719fd, 8165b73
Generating    comparison for : last_up

Generating    comparison for : last_updated.md  between: 6b141af, 2e18e95
Generating    comparison for : last_updated.md  between: e47b297, 6b141af
Generating    comparison for : last_updated.md  between: fffa892, e47b297
Generating    comparison for : last_updated.md  between: 32428fd, fffa892
Generating    comparison for : last_updated.md  between: 7259e91, 32428fd
Generating    comparison for : last_updated.md  between: 68f5b32, 7259e91
Generating    comparison for : last_updated.md  between: d3959ef, 68f5b32
Generating    comparison for : last_updated.md  between: 1e1c64d, d3959ef
Generating    comparison for : last_updated.md  between: a5ea8fc, 1e1c64d
Generating    comparison for : last_updated.md  between: a407e15, a5ea8fc
Generating    comparison for : last_updated.md  between: 2f19e8b, a407e15
Generating    comparison for : last_updated.md  between: a572fc9, 2f19e8b
Generating    comparison for : last_updated.md  between: 71be332, a572fc9
Generating    comparison for : last_up

In [ ]:
#hide
report = FileChangeHistoryReportForBranch(repository,b2,output_dir,styles_src=styles_src)
report.generate_report()

Generating csv comparison for : ACTION_APPEARANCES  between: 5c1c720, 07db82a
Generating csv comparison for : ACTION_APPEARANCES  between: b20832f, 5c1c720
Generating csv comparison for : ACTION_APPEARANCES  between: 1f9bfe5, b20832f
Generating csv comparison for : ACTION_APPEARANCES  between: f15aff0, 1f9bfe5
Generating csv comparison for : ACTION_APPEARANCES  between: 00e5899, f15aff0
Generating csv comparison for : ACTION_APPEARANCES  between: 778cc83, 00e5899
Generating csv comparison for : ACTION_APPEARANCES  between: 946a825, 778cc83
Generating csv comparison for : ACTION_APPEARANCES  between: 613c58f, 946a825
Generating csv comparison for : ACTION_APPEARANCES  between: 819edf0, 613c58f
Generating csv comparison for : ACTION_APPEARANCES  between: 39ccc54, 819edf0
Generating csv comparison for : ACTION_APPEARANCES  between: 2eb9c92, 39ccc54
Generating csv comparison for : ACTION_APPEARANCES  between: 1bc0591, 2eb9c92
Generating csv comparison for : ACTION_APPEARANCES  between: ff4

fatal: path 'database/ACTION_APPEARANCES.csv' exists on disk, but not in 'e7cf64d'
fatal: path 'database/ACTION_APPEARANCES.csv' exists on disk, but not in 'e7cf64d'


Command 'cd ../config_repository && git show e7cf64d:database/ACTION_APPEARANCES.csv' returned non-zero exit status 128.
Command 'cd ../config_repository && git show e7cf64d:database/ACTION_APPEARANCES.csv' returned non-zero exit status 128.
Generating csv comparison for : ACTION_APPEARANCES  between: 024888e, 4ee0e12
Generating csv comparison for : ACTION_APPEARANCES  between: 70c5de7, 024888e
Generating csv comparison for : ACTION_APPEARANCES  between: 7502a35, 70c5de7
Generating csv comparison for : ACTION_APPEARANCES  between: 0f9e182, 7502a35
Generating csv comparison for : ACTION_APPEARANCES  between: 0a9d357, 0f9e182
Generating csv comparison for : ACTION_APPEARANCES  between: db19172, 0a9d357
Generating csv comparison for : ACTION_APPEARANCES  between: 1070831, db19172
Generating csv comparison for : ACTION_APPEARANCES  between: b5fdbd6, 1070831
Generating csv comparison for : ACTION_APPEARANCES  between: f8c8982, b5fdbd6
Generating csv comparison for : ACTION_APPEARANCES  betw

fatal: path 'database/ACTION_DEFINITIONS.csv' exists on disk, but not in 'e7cf64d'
fatal: path 'database/ACTION_DEFINITIONS.csv' exists on disk, but not in 'e7cf64d'


Command 'cd ../config_repository && git show e7cf64d:database/ACTION_DEFINITIONS.csv' returned non-zero exit status 128.
Command 'cd ../config_repository && git show e7cf64d:database/ACTION_DEFINITIONS.csv' returned non-zero exit status 128.
Generating csv comparison for : ACTION_DEFINITIONS  between: 024888e, 2c79f03
Generating csv comparison for : ACTION_DEFINITIONS  between: 7502a35, 024888e
Generating csv comparison for : ACTION_DEFINITIONS  between: 6cfdd6b, 7502a35
Generating csv comparison for : ACTION_DEFINITIONS  between: 02bab03, 6cfdd6b
Generating csv comparison for : ACTION_DEFINITIONS  between: 0769608, 02bab03
Generating csv comparison for : ACTION_DEFINITIONS  between: 0a9d357, 0769608
Generating csv comparison for : ACTION_DEFINITIONS  between: b5fdbd6, 0a9d357
Generating csv comparison for : ACTION_DEFINITIONS  between: f8c8982, b5fdbd6
Generating csv comparison for : ACTION_DEFINITIONS  between: 83b06ca, f8c8982
Generating csv comparison for : ACTION_DEFINITIONS  betw

...

Generating    comparison for : last_updated.md  between: 7d402c6, 199cef4
Generating    comparison for : last_updated.md  between: 1c8bb18, 7d402c6
Generating    comparison for : last_updated.md  between: 39c0c61, 1c8bb18
Generating    comparison for : last_updated.md  between: ae9d762, 39c0c61
Generating    comparison for : last_updated.md  between: ec254f9, ae9d762
Generating    comparison for : last_updated.md  between: 91ace7b, ec254f9
Generating    comparison for : last_updated.md  between: 206168f, 91ace7b
Generating    comparison for : last_updated.md  between: 5840fe6, 206168f
Generating    comparison for : last_updated.md  between: 9ecc673, 5840fe6
Generating    comparison for : last_updated.md  between: 96b4d0c, 9ecc673
Generating    comparison for : last_updated.md  between: 9576532, 96b4d0c
Generating    comparison for : last_updated.md  between: b3b23c8, 9576532
Generating    comparison for : last_updated.md  between: 060de24, b3b23c8
Generating    comparison for : last_up

Generating    comparison for : last_updated.md  between: 6b9e7f2, efaf229
Generating    comparison for : last_updated.md  between: cfa6b60, 6b9e7f2
Generating    comparison for : last_updated.md  between: a34094b, cfa6b60
Generating    comparison for : last_updated.md  between: 56277e7, a34094b
Generating    comparison for : last_updated.md  between: 5ac6be3, 56277e7
Generating    comparison for : last_updated.md  between: 0c9e23f, 5ac6be3
Generating    comparison for : last_updated.md  between: 67a45f7, 0c9e23f
Generating    comparison for : last_updated.md  between: 7502a35, 67a45f7
Generating    comparison for : last_updated.md  between: 6cfdd6b, 7502a35
Generating    comparison for : last_updated.md  between: ccc10d0, 6cfdd6b
Generating    comparison for : last_updated.md  between: 94a1c85, ccc10d0
Generating    comparison for : last_updated.md  between: 97fdecc, 94a1c85
Generating    comparison for : last_updated.md  between: 1af6aa3, 97fdecc
Generating    comparison for : last_up

Generating    comparison for : last_updated.md  between: 3f703dc, c381ed4
Generating    comparison for : last_updated.md  between: b7e109a, 3f703dc
Generating    comparison for : last_updated.md  between: 2813dd1, b7e109a
Generating    comparison for : last_updated.md  between: 49b3ade, 2813dd1
Generating    comparison for : last_updated.md  between: a143485, 49b3ade
Generating    comparison for : last_updated.md  between: e9d3abf, a143485
Generating    comparison for : last_updated.md  between: dd4de49, e9d3abf
Generating    comparison for : last_updated.md  between: e67c7c8, dd4de49
Generating    comparison for : last_updated.md  between: 75c3215, e67c7c8
Generating    comparison for : last_updated.md  between: dde0244, 75c3215
Generating    comparison for : last_updated.md  between: 0a9d357, dde0244
Generating    comparison for : last_updated.md  between: d0a01df, 0a9d357
Generating    comparison for : last_updated.md  between: be32ea7, d0a01df
Generating    comparison for : last_up

Generating    comparison for : last_updated.md  between: 3a2ff00, 460f693
Generating    comparison for : last_updated.md  between: b70c1a4, 3a2ff00
Generating    comparison for : last_updated.md  between: 7a689a8, b70c1a4
Generating    comparison for : last_updated.md  between: cea523b, 7a689a8
Generating    comparison for : last_updated.md  between: f37e839, cea523b
Generating    comparison for : last_updated.md  between: 6d6f122, f37e839
Generating    comparison for : last_updated.md  between: 450408b, 6d6f122
Generating    comparison for : last_updated.md  between: 09239b7, 450408b
Generating    comparison for : last_updated.md  between: d0ce2a4, 09239b7
Generating    comparison for : last_updated.md  between: e868a5d, d0ce2a4
Generating    comparison for : last_updated.md  between: 3909d8e, e868a5d
Generating    comparison for : last_updated.md  between: 16e4e7c, 3909d8e
Generating    comparison for : last_updated.md  between: f7190b9, 16e4e7c
Generating    comparison for : last_up

Generating    comparison for : last_updated.md  between: a2e14aa, 94f0492
Generating    comparison for : last_updated.md  between: 5f7c2cd, a2e14aa
Generating    comparison for : last_updated.md  between: d761e36, 5f7c2cd
Generating    comparison for : last_updated.md  between: f126f83, d761e36
Generating    comparison for : last_updated.md  between: a008168, f126f83
Generating    comparison for : last_updated.md  between: a70bd13, a008168
Generating    comparison for : last_updated.md  between: 1a66215, a70bd13
Generating    comparison for : last_updated.md  between: 099ca0b, 1a66215
Generating    comparison for : last_updated.md  between: 28277ce, 099ca0b
Generating    comparison for : last_updated.md  between: 6ef5e43, 28277ce
Generating    comparison for : last_updated.md  between: c36ba35, 6ef5e43
Generating    comparison for : last_updated.md  between: 9c3b3eb, c36ba35
Generating    comparison for : last_updated.md  between: 23e2770, 9c3b3eb
Generating    comparison for : last_up

In [ ]:
#export
try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False
    
if __name__ == "__main__" and not IN_NOTEBOOK:


    ap = argparse.ArgumentParser()
    ap.add_argument("-a", "--branch1", required=True,
                help="name of the server to compare ")
    ap.add_argument("-b", "--branch2", required=False,
                help="name of the server to compare, if this is not supplied, a report with the history of branch1 will be generated",default='')
    ap.add_argument("-r", "--repository", required=True,
                help="directory of the repository")
    ap.add_argument("-o", "--outputdir", required=False, default="./",
                help="report output directory")
    ap.add_argument('-V', '--verbose', required=False,
                    help='add very chatty output, mainly for debug', default=False, action='store_true')
    ap.add_argument('-P', '--progress', required=False,
                    help='show progress', default=False, action='store_true')
    ap.add_argument('-s', '--stylesheets', required=True,
                    help='location of directory holding the stylesheets')

    args = vars(ap.parse_args())

    repository = args["repository"]
    output_dir = args["outputdir"]
    branch1 = args["branch1"]
    branch2 = args["branch2"]
    verbose = args['verbose']
    show_progress = args['progress']
    styles_src = args['stylesheets']
    
    if output_dir[-1] != '/':
        output_dir = output_dir + '/'
        
    if branch2 != '':  
        report = BranchComparisonReport(repository, branch1,branch2, output_dir,styles_src=styles_src)
        report.generate_report()
    
    report = FileChangeHistoryReportForBranch(repository,branch1,output_dir,styles_src=styles_src)
    report.generate_report()
    
    if branch2 != '':
        report = FileChangeHistoryReportForBranch(repository,branch2,output_dir,styles_src=styles_src)
        report.generate_report()


In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_csv_header_restore.ipynb.
Converted 01_convert_to_csv.ipynb.
Converted 02_initalise_repository.ipynb.
Converted 03_convert_files.ipynb.
Converted 04_generate_report.ipynb.
Converted index.ipynb.


In [ ]:
#hide
!echo '#!/usr/bin/env python2.7' > scripts/generate_report.py
!cat  config_tracker/generate_report.py >>scripts/generate_report.py
!chmod +x scripts/generate_report.py